# Validation of spike waveforms

author: steeve.laquitaine@sorbonne-university.fr  
date: 11.06.2025  
purpose: validate spike waveforms against in vivo  
Tested on: Ubuntu 24.04, 188 RAM, 32 cores (0.8-6GHz)  
Execution time: 

## Setup

Re-run from scratch:

0. Preprocess traces

1. Apply spike sorting

2. Extract spike waveforms

3. Setup and activate spikebias environment, run notebook with spikebias kernel

If data exists :

1. Dowload from dandi archive to datasets/01_intermediate
    - SortingExtractor for KS3

[TODO]: download intermediate data from GDrive -> upload to dandi archive


In [ ]:
%%time

# import libs
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import spikeinterface as si
import spikeinterface.extractors as se
from scipy.optimize import minimize
import spikeinterface.full as si_full
import seaborn as sns
import spikeinterface.postprocessing as spost
from concurrent.futures import ProcessPoolExecutor
from ipywidgets import interactive, widgets

# move to PROJECT PATH
proj_path = "/home/steeve/steeve/epfl/code/spikebias"
os.chdir(PROJ_PATH)

#from src.nodes.utils import get_config
#from src.nodes.postpro import waveform

# SETUP PARAMETERS
MS_BEFORE = 3           # ms
MS_AFTER = 3            # ms

# silico
#data_conf_sili, param_conf_sili = get_config("silico_neuropixels", "2023_10_18").values()
RAW_PATH = os.path.join(PROJ_PATH, "datasets/00_raw/recording_npx_spont/")
PREP_PATH = data_conf_sili["preprocessing"]["output"]["trace_file_path"]
WAVE_PATH = data_conf_sili["postprocessing"]["waveform"]["sorted"]["study"]
SORT_PATH = data_conf_sili["sorting"]["sorters"]["kilosort3"]["output"]

# set parameters
PAULK_PERIOD = 0.55
SUN_PERIOD = 4
OUR_PERIOD = 6

# FIGURE SETTINGS
COLOR_VIVO = (0.7, 0.7, 0.7)
COLOR_SILI = (0.84, 0.27, 0.2)
COLOR_STIM = (0.6, 0.75, 0.1)
BOX_ASPECT = 1                  # square fig
FIG_SIZE = (4,4)
N_MAJOR_TICKS = 6
N_MINOR_TICKS = 12
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 24
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['xtick.major.size'] = 7
plt.rcParams['ytick.major.size'] = 7
plt.rcParams['ytick.major.width'] = 1
legend_cfg = {"frameon": False, "handletextpad": 0.1}
savefig_cfg = {"transparent":True}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


FileNotFoundError: [Errno 2] No such file or directory: '/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/spikebias/'